In [53]:
import configparser
import tweepy

# read config
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

# authenticate
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)

In [54]:
hashtags = ['#ukrainewar', '#war', '#army', '#military', '#kiev', '#ua', '#specialforces', '#donbass', '#donbasswar', '#airsoft', '#nomockal', '#warukraine', '#tactics', '#azovsea', '#militarystile', '#azov', '#russia', '#donetsk', '#soldiers', '#ukrainenews', '#odessa', '#ukrainianarmy', '#lviv', '#victory', '#nato', '#kyiv', '#militaryukraine', '#news']
#hashtags = ['#ukrainewar', '#war', '#army', '#military', '#kiev', '#ua', '#specialforces', '#donbass', '#donbasswar', '#nomockal', '#warukraine', '#azovsea', '#azov', '#russia', '#donetsk', '#soldiers', '#ukrainenews', '#odessa', '#ukrainianarmy', '#lviv', '#victory', '#nato', '#kyiv', '#militaryukraine']
num_tweets = 20000
date_since = '2022-02-22'
date_until = '2022-04-20'
query = ' OR '.join(hashtags)
query = query + ' -filter:retweets'

tweets = tweepy.Cursor(api.search_tweets,
                        q=query,
                        tweet_mode='extended').items(num_tweets)

In [55]:
list_tweets = [tweet for tweet in tweets]

Rate limit reached. Sleeping for: 815
Rate limit reached. Sleeping for: 812
Rate limit reached. Sleeping for: 817
Rate limit reached. Sleeping for: 818
Rate limit reached. Sleeping for: 817
Rate limit reached. Sleeping for: 821
Rate limit reached. Sleeping for: 820


In [61]:
import pandas as pd

db = pd.DataFrame(columns=['id',
                            'user_id',
                            'username',
                            'location',
                            'geoid',
                            'text',
                            'language',
                            'date',
                            'hashtags',
                            'retweet_count',
                            'replies'])

for tweet in list_tweets:
    id = tweet.id
    user_id = tweet.user.id
    username = tweet.user.screen_name
    location = tweet.user.location
    geoid = tweet.geo
    language = tweet.lang
    created_at = tweet.created_at
    hashtags_tweet = tweet.entities['hashtags']
    retweet_count = tweet.retweet_count
    replies = tweet.in_reply_to_status_id
    
    text = tweet.full_text
    hashtext = list()
    for j in range(0, len(hashtags_tweet)):
        hashtext.append(hashtags_tweet[j]['text'])
        ith_tweet = [id, user_id, username, location, geoid,
                        text, language, created_at, hashtext, retweet_count, replies]
        db.loc[len(db)] = ith_tweet

filename = 'tweets_2022-05-08.csv'

db = db.drop_duplicates(subset=['id'])

db.to_csv(filename)